<a href="https://colab.research.google.com/github/garthajon/QuantFinanceIntro/blob/main/zcore_stock_screen_ftse100_UK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta

# ------------------ Fetch FTSE 100 tickers ------------------
ftse100_url = "https://en.wikipedia.org/wiki/FTSE_100_Index"
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36"
    )
}
resp = requests.get(ftse100_url, headers=headers)
tables = pd.read_html(resp.text)

ftse100_table = None

for t in tables:
    # Flatten MultiIndex columns if needed
    if isinstance(t.columns, pd.MultiIndex):
        t.columns = [" ".join(map(str, col)).strip() for col in t.columns.values]
    else:
        t.columns = t.columns.astype(str)

    # Find table that contains something like "EPIC", "Ticker", or "Symbol"
    if any(c.lower() in ["epic", "ticker", "symbol"] for c in t.columns.str.lower()):
        ftse100_table = t
        break

if ftse100_table is None:
    raise ValueError("Could not find FTSE 100 constituents table on Wikipedia. Structure may have changed.")

# Identify the correct ticker column
ticker_col = next(
    (col for col in ftse100_table.columns if col.lower() in ["epic", "ticker", "symbol"]),
    None
)
if ticker_col is None:
    raise ValueError("No 'EPIC' or 'Ticker' column found in FTSE 100 table.")

tickers = ftse100_table[ticker_col].dropna().astype(str).str.strip().str.upper()

# Append .L suffix for Yahoo Finance (London-listed)
tickers = [f"{t}.L" for t in tickers]

# ------------------ Fetch daily close price data ------------------
end_date = datetime.today()
start_date = end_date - timedelta(weeks=52*1)  # 1 year

print(f"Downloading {len(tickers)} tickers (1 year daily)...")
data = yf.download(tickers, start=start_date, end=end_date, interval="1d", progress=False)["Close"]

# Drop tickers with incomplete data
data = data.dropna(axis=1, how="any")
available_tickers = list(data.columns)
print(f"{len(available_tickers)} tickers with complete data.\n")

# ------------------ Compute weekly annualized volatility ------------------
returns = data.pct_change().dropna()
weekly_std = returns.resample("W").std() * np.sqrt(252)  # annualized volatility

# ------------------ Compute Z-score (most recent week) ------------------
zscore_params = {}
for ticker in available_tickers:
    mu = weekly_std[ticker].mean()
    sigma = weekly_std[ticker].std(ddof=0)
    zscore_params[ticker] = (mu, sigma)

latest_weekly_std = weekly_std.iloc[-1]

zscores = pd.Series({
    t: (latest_weekly_std[t] - zscore_params[t][0]) / zscore_params[t][1]
    for t in available_tickers
})

# ------------------ Screener: only extreme tickers ------------------
extreme_z = zscores[(zscores > 2) | (zscores < -2)].sort_values(ascending=False)

# ------------------ Display screener ------------------
print("\n=== FTSE 100 STOCK SCREENER: Z > 2 OR Z < -2 ===\n")
print(extreme_z.to_frame("Z-Score"))


/tmp/ipython-input-3088203546.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)
/tmp/ipython-input-3088203546.py:53: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date, interval="1d", progress=False)["Close"]


94 tickers with complete data.


=== FTSE 100 STOCK SCREENER: Z > 2 OR Z < -2 ===

         Z-Score
AZN.L   3.238027
TSCO.L  2.587613
GSK.L   2.197065
